## ツールやライブラリ（LangChain）との統合を用いた計装

必要なライブラリをダウンロードします。

In [1]:
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


実行に必要な環境変数を読み込みます。

In [2]:
import os, warnings, time
from dotenv import load_dotenv

load_dotenv("../.env")

# おまじない
warnings.filterwarnings("ignore")

endpoint = "http://localhost:3000"
public_key = os.getenv("PUBLIC_KEY")
secret_key = os.getenv("SECRET_KEY")

Langfuseが提供するLangChainのCallback実装を初期化します。

In [3]:
from langfuse.callback import CallbackHandler

# 環境変数（LANGFUSE_SECRET_KEY, LANGFUSE_PUBLIC_KEY, LANGFUSE_HOST）を定義でも良い
callback_handler = CallbackHandler(
    public_key=public_key,
    secret_key=secret_key,
    host=endpoint,
)

簡易的なRAGアプリケーションを構築します。詳細は、[chapter03-03-simple-llm-app-with-rag.ipynb](./chapter03-03-simple-llm-app-with-rag.ipynb)を参照ください。

In [8]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS

xql_doc = WebBaseLoader("https://techbookfest.org/product/vwEgK9fAmzRphNukv4E83P?productVariantID=b6iAh0AVyEs4hCUczPiy").load()

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.embeddings import OpenAIEmbeddings
    openai_api_key = os.getenv("OPENAI_API_KEY")
    embeddings = OpenAIEmbeddings(api_key=openai_api_key)
else:
    from langchain_cohere.embeddings import CohereEmbeddings
    cohere_api_key = os.getenv("COHERE_API_KEY")
    embeddings = CohereEmbeddings(
        cohere_api_key=cohere_api_key,
        model="embed-multilingual-v3.0"
    )

xql_db = FAISS.from_documents(
    documents=xql_doc,
    embedding=embeddings
)
xql_retriever = xql_db.as_retriever()

prompt_template = PromptTemplate.from_template(template="""
あなたは有能なアシスタントです。
以下のコンテキストに基づいて質問に対する回答を作成してください。
分からない場合は分からないと回答してください。

## コンテキスト

{context}

## 質問

{question}
""")

if os.getenv("COHERE_API_KEY") == None:
    from langchain_openai.chat_models import ChatOpenAI
    openai_api_key = os.getenv("OPENAI_API_KEY")
    model = "gpt-4o-mini"
    chat = ChatOpenAI(api_key=openai_api_key, model=model)
else:
    from langchain_cohere.chat_models import ChatCohere
    cohere_api_key = os.getenv("COHERE_API_KEY")
    model = "command-r-plus"
    chat = ChatCohere(cohere_api_key=cohere_api_key, model=model)

chain = (
    {"question": RunnablePassthrough(), "context": xql_retriever}
    | prompt_template
    | chat
    | StrOutputParser()
)

from langfuse.decorators import observe,langfuse_context

@observe(name="meet the bigbaby")
def meet_the_bigbaby():
    print("Meet the BigBaBy!")

# RAGを行うChainを定義
response = chain.invoke(
    input="xQL完全ガイドってなに？",
    # LangChainのCallback実装をChain実行時のConfigに渡す
    config={"callbacks": [callback_handler]}
)

meet_the_bigbaby()

langfuse_context.flush()

print(response)

validate_env
generate
{'message': "\nあなたは有能なアシスタントです。\n以下のコンテキストに基づいて質問に対する回答を作成してください。\n分からない場合は分からないと回答してください。\n\n## コンテキスト\n\n[Document(metadata={'source': 'https://techbookfest.org/product/vwEgK9fAmzRphNukv4E83P?productVariantID=b6iAh0AVyEs4hCUczPiy', 'title': '俺たちのxQL完全ガイド - PromQL / LogQL / TraceQL 編 -：〆のラーメンまである倶楽部', 'description': 'Grafanaでクエリを書く際、雰囲気で書いていませんか？\\n本書は、そんな読者に向けた完全ガイドです。\\n\\n本書では、OSSのモニタリングツールであるPrometheus / Grafana Loki / Grafana Tempoで用いられるクエリ言語、PromQL / LogQL / TraceQL に焦点を当てて解説します。これらのクエリ言語を使いこなすことで、計測したテレメトリーシグナルの中から自分たちが欲しいメトリクス、ログ、トレースをより簡単に探索できます。\\n\\n本書の構成として、メトリクス、ログ、トレースの概要や各クエリ言語について説明した後、いくつかの参考問題を用意しています。また、各クエリ言語を読者の手元でも動かせるように、サンプルアプリとDocker環境を整備しています。読むだけではなく、実際に手を動かしながら学び進めることもできます。\\n\\n本書を通じて、PromQL / LogQL / TraceQLの理解を深め、効果的に活用できるようになることを願っています。また、本書がこれらのクエリ言語について学ぶ一助となり、システムのオブザーバビリティを高め、クラウドネイティブなアプリケーションを運用するエンジニアの最高の武器となれば幸いです。\\n\\nそれでは、PromQL / LogQL / TraceQL の世界に一緒に旅立ちましょう！', 'language': 'ja'}, page_content='俺たちのxQL完全ガイド - PromQL / 